In [ ]:
#NOTE: When committing changes to this notebook, please restart the kernel and clear outputs.
#      this will reduce the odds of a merge conflict

import sys, os
import numpy as np
# import h5py

from pygama.dsp.ProcessingChain import ProcessingChain
from pygama.dsp.processors import *
from pygama.dsp.units import *

from pygama.io import lh5

# CONSTANT VARIABLES TO SET UP CONFIG
# with h5py.File("/Volumes/LaCie/Data/CAGE/pygama_raw/raw_run11.lh5", 'r') as f:
#     print(f.keys())
filename = os.path.expandvars("t1_run1687.lh5")
groupname = "ORSIS3302DecoderForEnergy"
nentries = 1000 # Number of entries in the file to process
start_entry = 0 # First entry to process
verbosity = 2
wfblock = 8

In [ ]:
# Set up file I/O
lh5_in = lh5.Store()
data = lh5_in.read_object(groupname, filename, n_rows=nentries, start_row=start_entry)

wf_in = data['waveform']['values'].nda
dt = data['waveform']['dt'].nda[0] * unit_parser.parse_unit(data['waveform']['dt'].attrs['units'])

# Set up processing chain
proc = ProcessingChain(block_width=wfblock, clock_unit=dt, verbosity=verbosity)
proc.add_input_buffer("wf", wf_in, dtype='float32')

# Basic Filters
proc.add_processor(mean_stdev, "wf[0:1000]", "bl", "bl_sig")
proc.add_processor(np.subtract, "wf", "bl", "wf_blsub")
proc.add_processor(pole_zero, "wf_blsub", 70*us, "wf_pz")
proc.add_processor(trap_norm, "wf_pz", 10*us, 5*us, "wf_trap")
proc.add_processor(asymTrapFilter, "wf_pz", 0.05*us, 2*us, 4*us, "wf_atrap")

# Timepoint calculation
proc.add_processor(np.argmax, "wf_blsub", 1, "t_max", signature='(n),()->()', types=['fi->i'])
proc.add_processor(time_point_frac, "wf_blsub", 0.95, "t_max", "tp_95")
proc.add_processor(time_point_frac, "wf_blsub", 0.8, "t_max", "tp_80")
proc.add_processor(time_point_frac, "wf_blsub", 0.5, "t_max", "tp_50")
proc.add_processor(time_point_frac, "wf_blsub", 0.2, "t_max", "tp_20")
proc.add_processor(time_point_frac, "wf_blsub", 0.05, "t_max", "tp_05")
proc.add_processor(time_point_thresh, "wf_atrap[0:2000]", 0, "tp_0")

# Energy calculation
proc.add_processor(np.amax, "wf_trap", 1, "trapEmax", signature='(n),()->()', types=['fi->f'])
proc.add_processor(fixed_time_pickoff, "wf_trap", "tp_0+(5*us+9*us)", "trapEftp")
proc.add_processor(trap_pickoff, "wf_pz", 1.5*us, 0, "tp_0", "ct_corr")

# Current calculation
proc.add_processor(avg_current, "wf_pz", 10, "curr(len(wf_pz)-10, f)")
proc.add_processor(np.amax, "curr", 1, "curr_amp", signature='(n),()->()', types=['fi->f'])
proc.add_processor(np.divide, "curr_amp", "trapEftp", "aoe")

# DCR calculation: use slope using 1000 samples apart and averaging 200
# samples, with the start 1.5 us offset from t0
proc.add_processor(trap_pickoff, "wf_pz", 200, 1000, "tp_0+1.5*us", "dcr_unnorm")
proc.add_processor(np.divide, "dcr_unnorm", "trapEftp", "dcr")

# Tail slope. Basically the same as DCR, except with no PZ correction
proc.add_processor(linear_fit, "wf_blsub[3000:]", "wf_b", "wf_m")
proc.add_processor(np.divide, "-wf_b", "wf_m", "tail_rc")


In [ ]:
# Get the outputs of interest
wf = proc.get_output_buffer("wf_blsub")
trap = proc.get_output_buffer("wf_trap")
atrap = proc.get_output_buffer("wf_atrap")

tmax = proc.get_output_buffer("t_max")
tp95 = proc.get_output_buffer("tp_95")
tp80 = proc.get_output_buffer("tp_80")
tp50 = proc.get_output_buffer("tp_50")
tp20 = proc.get_output_buffer("tp_20")
tp05 = proc.get_output_buffer("tp_05")
tp0 = proc.get_output_buffer("tp_0")

trapEmax = proc.get_output_buffer("trapEmax")
trapEftp = proc.get_output_buffer("trapEftp")
ctcorr = proc.get_output_buffer("ct_corr")

curr = proc.get_output_buffer("curr")
AoverE = proc.get_output_buffer("aoe")

wfindex = np.arange(trap.shape[-1])

# Execute analysis
proc.execute()


In [ ]:
# Set index counter. Rerun this box to start over the file...
wfnum = 0

In [ ]:
import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = [10, 5]

# Draw Waveforms and timepoints
plt.plot(wfindex, wf[wfnum])
plt.plot(wfindex, atrap[wfnum])
plt.vlines(tp95[wfnum], 0, trapEmax[wfnum])
plt.vlines(tp80[wfnum], 0, trapEmax[wfnum])
plt.vlines(tp50[wfnum], 0, trapEmax[wfnum])
plt.vlines(tp20[wfnum], 0, trapEmax[wfnum])
plt.vlines(tp05[wfnum], 0, trapEmax[wfnum])
plt.vlines(tp0[wfnum], 0, trapEmax[wfnum])

# plt.scatter(wfindex, curr[wfnum])
plt.hlines(trapEmax[wfnum], wfindex[0], wfindex[-1], color='r')
plt.hlines(trapEftp[wfnum], wfindex[0], wfindex[-1], linestyle='dotted', color='b')
# plt.hlines(curr_amp[wfnum], wfindex[0], wfindex[-1])
plt.xlim(900, 1400)
plt.xlabel('Time (10 ns)')
plt.ylabel('ADC')
plt.show()
wfnum+=1

In [ ]:
# Energy Plots
plt.hist(trapEmax, 100, (0, 10000))
plt.xlabel('Energy (ADC)')
plt.show()

plt.hist(trapEftp, 100, (0, 10000))
plt.xlabel('Energy (ADC)')
plt.show()

plt.hist(trapEmax, 50, (8000, 9000))
plt.hist(trapEftp, 50, (8000, 9000))
plt.xlabel('Energy (ADC)')
plt.show()

plt.scatter(trapEftp, ctcorr)
plt.xlim(8300, 8450)
plt.xlabel('Energy (ADC)')
plt.ylabel('Charge Trapping Correction (arb)')
plt.show()

In [ ]:
# A/E plot
plt.scatter(trapEftp, AoverE, marker=".")
plt.ylim(0, 0.1)
plt.xlabel('Energy (ADC)')
plt.ylabel("A/E (arb)")
plt.show()